# This notebook show my issue with the calculation time of the enrichment function

In [8]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [9]:
import greatpy as great
import pandas as pd

In [10]:
test = pd.read_csv("../data/human/test_genomic_region.bed",sep="\t",comment="#",
                names=["Chr", "Chr_Start", "Chr_End"],dtype={"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64"})

df_10_row = test[:50]

In [11]:
%lprun -f great.tl.enrichment great.tl.enrichment(df_10_row,"../data/human/regulatory_domain.bed","../data/human/chr_size.bed","../data/human/ontologies.csv",True,True,0.05,(0,0))

Timer unit: 1e-06 s

Total time: 29.7136 s
File: /home/tom/.local/lib/python3.8/site-packages/greatpy/tl/basic.py
Function: enrichment at line 282

Line #      Hits         Time  Per Hit   % Time  Line Contents
   282                                           def enrichment(test:str or pd.DataFrame,regdom_file,chr_size_file,annotation,binom=True,hypergeom=True,alpha=0.05,correction=("fdr",0.05),sort_by=None): 
   283                                               # Data import 
   284         1          5.0      5.0      0.0      if not binom and not hypergeom : 
   285                                                   return False
   286                                               
   287         2      72358.0  36179.0      0.2      regdom = pd.read_csv(regdom_file,sep="\t",comment="#",
   288         1          4.0      4.0      0.0                      names=["Chr", "Chr_Start", "Chr_End","Name","tss","Strand"],dtype={"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64","Name":"

Here we can see that the calculation time of the enrichment function is very high due to : 

line 341 (50 %) in the https://github.com/theislab/greatpy/blob/correction/greatpy/tl/basic.py with k_binom = number_of_hit(test,curr_regdom) => The number_of_hit function is used to calculate the number of genomic region hit with each GO term 

line 351 (30 %) https://github.com/theislab/greatpy/blob/correction/greatpy/tl/basic.py with dict.update with the values of binom p-value and hypergeom p-value 